In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib
import geopandas
import os.path

In [ ]:
f = os.path.expanduser("~/Documents/Vodacom/data/df_raw_final.pkl")
#reading from datafile
df = pd.read_pickle(f)
#datafile = df

In [ ]:
df

In [ ]:
df.D_LEARNERS_KM2.mean()
df.D_MIGR_IN_KM2.max()